In [1]:
%cd ~/austin_big_vision
import io
import jax
import torch
import numpy as np
import ml_collections
import jax.numpy as jnp

import open_clip
from big_vision import utils as u
from big_vision.models.vit import scan_to_pyloop

%load_ext autoreload
%autoreload 2
save_bv_vit = False
np_save_path = "/home/austinwang/ckpts_npz/bv_siglip.npz"


/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/IPython/core/magics/osm.py:393: UserWarning: This is now an optional IPython functionality, using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})
/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/austinwang/austin_big_vision


/mnt/vlm-pd/miniconda3/envs/vlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-07-18 13:13:02.177266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-18 13:13:02.199512: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-18 13:13:02.206412: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-18 13:13:03.148283: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning:

In [2]:
# load json file from google cloud storage
import json
import tensorflow as tf
gs_path = "gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/siglip_parallel_bs16384_warm0.03_lr1e-3_wd1e-4_bf16_qknorm-F_b2-0.95_12lyr_06-24_2019/config.json"

with tf.io.gfile.GFile(gs_path, "r") as f:
    config = json.load(f)
config.keys()

dict_keys(['input', 'total_steps', 'init_shapes', 'init_types', 'log_training_steps', 'ckpt_steps', 'model_name', 'model_load', 'model', 'loss_fn', 'optax_name', 'optax', 'mesh', 'sharding_strategy', 'lr', 'wd', 'schedule', 'grad_clip_norm', 'evals', 'wandb'])

# big_vision ViT

In [3]:
import big_vision.models.vit as model_mod
model_cfg = ml_collections.ConfigDict()
model_cfg.num_classes = None
model_cfg.patch_size = (16, 16)
model_cfg.width = 768
model_cfg.depth = 12
model_cfg.mlp_dim = None  # Defaults to 4x input dim
model_cfg.num_heads = 12
model_cfg.posemb = "learn"  # Can also be "sincos2d"
model_cfg.rep_size = False
model_cfg.dropout = 0.0
model_cfg.pool_type = "map"  # Can also be "map" or "tok"
model_cfg.head_zeroinit = True
model_cfg.mask = None
model_cfg.normalize_qk = False
model_cfg.scan = True
model_cfg.remat_policy = "nothing_saveable"
model_cfg.dtype_mm = "bfloat16"

bv_model = model_mod.Model(**model_cfg)

# cappa_ckpt = 'gs://us-central2-storage/tensorflow_datasets/cappa_bs16384_warm0.02_lr1e-3_wd1e-4_bf16_b2-0.95_6lyr_06-15_2102/checkpoint.bv-000183105'
# cappa_path = cappa_ckpt+':encoder'
siglip_ckpt = 'gs://us-central2-storage/tensorflow_datasets/vit-b-16_3b_pretraining/siglip_replication_pod_04-11_2247/checkpoint.bv-000183105'
siglip_path = siglip_ckpt+':img'

init_params = None
params = model_mod.load(init_params, siglip_path, model_cfg)
jax.tree_map(jnp.shape, params)

I0000 00:00:1721308386.834536 3665871 gcs_resource.cc:109] Using default AdmissionQueue with limit 32
I0000 00:00:1721308386.842992 3667264 google_auth_provider.cc:180] Running on GCE, using service account 373177222751-compute@developer.gserviceaccount.com
/tmp/ipykernel_3665871/93864302.py:29: DeprecationWarning: jax.tree_map is deprecated: use jax.tree.map (jax v0.4.25 or newer) or jax.tree_util.tree_map (any JAX version).
  jax.tree_map(jnp.shape, params)


{'MAPHead_0': {'LayerNorm_0': {'bias': (768,), 'scale': (768,)},
  'MlpBlock_0': {'Dense_0': {'bias': (3072,), 'kernel': (768, 3072)},
   'Dense_1': {'bias': (768,), 'kernel': (3072, 768)}},
  'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 64),
    'kernel': (768, 12, 64)},
   'out': {'bias': (768,), 'kernel': (12, 64, 768)},
   'query': {'bias': (12, 64), 'kernel': (768, 12, 64)},
   'value': {'bias': (12, 64), 'kernel': (768, 12, 64)}},
  'probe': (1, 1, 768)},
 'Transformer': {'encoder_norm': {'bias': (768,), 'scale': (768,)},
  'encoderblock': {'LayerNorm_0': {'bias': (12, 768), 'scale': (12, 768)},
   'LayerNorm_1': {'bias': (12, 768), 'scale': (12, 768)},
   'MlpBlock_0': {'Dense_0': {'bias': (12, 3072), 'kernel': (12, 768, 3072)},
    'Dense_1': {'bias': (12, 768), 'kernel': (12, 3072, 768)}},
   'MultiHeadDotProductAttention_0': {'key': {'bias': (12, 12, 64),
     'kernel': (12, 768, 12, 64)},
    'out': {'bias': (12, 768), 'kernel': (12, 12, 64, 768)},
    'query': {'

In [4]:
if save_bv_vit:
    # save big_vision ckpt as npz file
    ckpt = {'params': {'img': scan_to_pyloop(params)}}
    names_and_vals, _ = u.tree_flatten_with_names(ckpt)
    io_buffer = io.BytesIO()
    np.savez(io_buffer, **{k: v for k, v in names_and_vals})
    with open(np_save_path, "wb") as f:
        f.write(io_buffer.getvalue())

    # load npz file
    big_vision_vit = np.load(np_save_path)
    for key in big_vision_vit.keys():
        print(key, big_vision_vit[key].shape)

# open_clip

In [5]:
@torch.no_grad()
def load_any_bv_vit(model, checkpoint_path, strict=True):
    print(f"HELLO! I changed the load_checkpoint function!!!!")
    from timm.layers import resample_patch_embed, resample_abs_pos_embed

    def _n2p(w, t=True):
        if w.ndim == 4 and w.shape[0] == w.shape[1] == w.shape[2] == 1:
            w = w.flatten()
        if t:
            if w.ndim == 4:
                w = w.transpose([3, 2, 0, 1])
            elif w.ndim == 3:
                w = w.transpose([2, 0, 1])
            elif w.ndim == 2:
                w = w.transpose([1, 0])
        return torch.from_numpy(w)

    w = np.load(checkpoint_path)
    interpolation = 'bilinear'
    antialias = False
    
    module = model.visual.trunk
    prefix = 'params/img/'
    embed_conv_w = _n2p(w[f'{prefix}embedding/kernel'])
    if embed_conv_w.shape[-2:] != module.patch_embed.proj.weight.shape[-2:]:
        embed_conv_w = resample_patch_embed(
            embed_conv_w,
            module.patch_embed.proj.weight.shape[-2:],
            interpolation=interpolation,
            antialias=antialias,
            verbose=True,
        )
    module.patch_embed.proj.weight.copy_(embed_conv_w)
    module.patch_embed.proj.bias.copy_(_n2p(w[f'{prefix}embedding/bias']))

    if module.cls_token is not None:
        module.cls_token.copy_(_n2p(w[f'{prefix}cls'], t=False))

    pos_embed_w = _n2p(w[f'{prefix}pos_embedding'], t=False)
    if pos_embed_w.shape != module.pos_embed.shape:
        assert False, f'{pos_embed_w.shape}, {module.pos_embed.shape}'
        num_prefix_tokens = 0 if getattr(module, 'no_embed_class', False) else getattr(module, 'num_prefix_tokens', 1)
        pos_embed_w = resample_abs_pos_embed(  # resize pos embedding when different size from pretrained weights
            pos_embed_w,
            new_size=module.patch_embed.grid_size,
            num_prefix_tokens=num_prefix_tokens,
            interpolation=interpolation,
            antialias=antialias,
            verbose=True,
        )
    module.pos_embed.copy_(pos_embed_w)

    mha_sub, b_sub, ln1_sub = (0, 0, 1)
    for i, block in enumerate(module.blocks.children()):
        block_prefix = f'{prefix}Transformer/encoderblock_{i}/'
        mha_prefix = block_prefix + f'MultiHeadDotProductAttention_{mha_sub}/'
        block.norm1.weight.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/scale']))
        block.norm1.bias.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/bias']))
        block.attn.qkv.weight.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/kernel'], t=False).flatten(1).T for n in ('query', 'key', 'value')]))
        block.attn.qkv.bias.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/bias'], t=False).reshape(-1) for n in ('query', 'key', 'value')]))
        block.attn.proj.weight.copy_(_n2p(w[f'{mha_prefix}out/kernel']).flatten(1))
        block.attn.proj.bias.copy_(_n2p(w[f'{mha_prefix}out/bias']))
        for r in range(2):
            getattr(block.mlp, f'fc{r + 1}').weight.copy_(_n2p(w[f'{block_prefix}MlpBlock_{b_sub}/Dense_{r}/kernel']))
            getattr(block.mlp, f'fc{r + 1}').bias.copy_(_n2p(w[f'{block_prefix}MlpBlock_{b_sub}/Dense_{r}/bias']))
        block.norm2.weight.copy_(_n2p(w[f'{block_prefix}LayerNorm_{ln1_sub}/scale']))
        block.norm2.bias.copy_(_n2p(w[f'{block_prefix}LayerNorm_{ln1_sub}/bias']))

    module.norm.weight.copy_(_n2p(w[f'{prefix}Transformer/encoder_norm/scale']))
    module.norm.bias.copy_(_n2p(w[f'{prefix}Transformer/encoder_norm/bias']))

    if module.attn_pool is not None:
        block_prefix = f'{prefix}MAPHead_0/'
        mha_prefix = block_prefix + f'MultiHeadDotProductAttention_0/'
        module.attn_pool.latent.copy_(_n2p(w[f'{block_prefix}probe'], t=False))
        module.attn_pool.q.weight.copy_(_n2p(w[f'{mha_prefix}query/kernel'], t=False).flatten(1).T)
        module.attn_pool.q.bias.copy_(_n2p(w[f'{mha_prefix}query/bias'], t=False).reshape(-1))
        module.attn_pool.kv.weight.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/kernel'], t=False).flatten(1).T for n in ('key', 'value')]))
        module.attn_pool.kv.bias.copy_(torch.cat([
            _n2p(w[f'{mha_prefix}{n}/bias'], t=False).reshape(-1) for n in ('key', 'value')]))
        module.attn_pool.proj.weight.copy_(_n2p(w[f'{mha_prefix}out/kernel']).flatten(1))
        module.attn_pool.proj.bias.copy_(_n2p(w[f'{mha_prefix}out/bias']))
        module.attn_pool.norm.weight.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/scale']))
        module.attn_pool.norm.bias.copy_(_n2p(w[f'{block_prefix}LayerNorm_0/bias']))
        for r in range(2):
            getattr(module.attn_pool.mlp, f'fc{r + 1}').weight.copy_(_n2p(w[f'{block_prefix}MlpBlock_0/Dense_{r}/kernel']))
            getattr(module.attn_pool.mlp, f'fc{r + 1}').bias.copy_(_n2p(w[f'{block_prefix}MlpBlock_0/Dense_{r}/bias']))

open_clip.convert.load_big_vision_weights = load_any_bv_vit


In [49]:
model_type = "ViT-B-16-SigLIP"
open_clip_model, processor = open_clip.create_model_from_pretrained(
    model_type,
    pretrained=np_save_path,
    precision='bf16',
)
open_clip_model

HELLO! I changed the load_checkpoint function!!!!


CustomTextCLIP(
  (visual): TimmModel(
    (trunk): VisionTransformer(
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
        (norm): Identity()
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (patch_drop): Identity()
      (norm_pre): Identity()
      (blocks): Sequential(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (q_norm): Identity()
            (k_norm): Identity()
            (attn_drop): Dropout(p=0.0, inplace=False)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
          )
          (ls1): Identity()
          (drop_path1): Identity()
          (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (mlp): Mlp(
            (fc1): Linear(in_features=768

# big_vision vs. open_clip

In [54]:
num_tests = 10

similarity_list = []
for i in range(num_tests):
    print(f"test {i}")
    # initialize dummy image with same values
    open_clip_dummy_img = torch.randn(1, 3, 224, 224).bfloat16()
    bv_dummy_img = jnp.array(open_clip_dummy_img.permute(0, 2, 3, 1).numpy())
    bv_logits = bv_model.apply({"params":params}, bv_dummy_img)[0][0]
    open_clip_logits = open_clip_model(open_clip_dummy_img)[0][0]
    # l2 normalization
    bv_logits = bv_logits / jnp.linalg.norm(bv_logits)
    open_clip_logits = open_clip_logits / torch.norm(open_clip_logits)
    # cosine similarity
    similarity = jnp.dot(bv_logits, open_clip_logits.detach().numpy())
    similarity_list.append(similarity)
sum(similarity_list) / num_tests

test 0


TypeError: Got unsupported ScalarType BFloat16